<a href="https://colab.research.google.com/github/Deepanshu02Gupta/Netflix-Movie-Recommender/blob/main/Netflix_Movie_Recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing Libraries**

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tkinter import *
import pickle
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import plotly as plotly
from scipy.stats import norm
from sklearn.feature_extraction.text import TfidfVectorizer

# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## **Importing DataSet**





In [ ]:
datafr = pd.read_csv('netflix_titles1.csv')

## **Data Manipulation**

In [ ]:
datafr.isnull().sum()

show_id            0
type               0
title              0
director        2631
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

**Creating Another column for Month and Year**

In [ ]:
datafr["date_added"] = pd.to_datetime(datafr['date_added'])
datafr['year_added'] = datafr['date_added'].dt.year
datafr['month_added'] = datafr['date_added'].dt.month

In [ ]:
datafr.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,year_added,month_added
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",2021.0,9.0
1,s2,TV Show,Blood & Water,John L'Ecuyer,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",2021.0,9.0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,2021.0,9.0
3,s4,TV Show,Jailbirds New Orleans,Audrey Estrougo,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",2021.0,9.0
4,s5,TV Show,Kota Factory,Raghav Subbu,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,2021.0,9.0


## **Data Visualisation**

**Create a Bar Plot**

In [ ]:
plt.bar

<function matplotlib.pyplot.bar>

**Create new column for count of movies and tvshows by month**

In [ ]:
col = 'month_added'
vc1 = datafr[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

In [ ]:
vc1

,month_added,count,percent
7,1.0,738,8.389224
11,2.0,563,6.399909
6,3.0,742,8.434694
3,4.0,764,8.684779
10,5.0,632,7.184267
8,6.0,728,8.275548
0,7.0,827,9.400932
5,8.0,755,8.582471
2,9.0,770,8.752984
4,10.0,760,8.639309


**Creating graph objects to plot**

In [ ]:
trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
data = [trace1]
layout = go.Layout(title="In which month, the conent is added the most?", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

**Geographical Visualisation and bar plot, counting content country wise**

In [ ]:
country_codes = {'afghanistan': 'AFG',
 'albania': 'ALB',
 'algeria': 'DZA',
 'american samoa': 'ASM',
 'andorra': 'AND',
 'angola': 'AGO',
 'anguilla': 'AIA',
 'antigua and barbuda': 'ATG',
 'argentina': 'ARG',
 'armenia': 'ARM',
 'aruba': 'ABW',
 'australia': 'AUS',
 'austria': 'AUT',
 'azerbaijan': 'AZE',
 'bahamas': 'BHM',
 'bahrain': 'BHR',
 'bangladesh': 'BGD',
 'barbados': 'BRB',
 'belarus': 'BLR',
 'belgium': 'BEL',
 'belize': 'BLZ',
 'benin': 'BEN',
 'bermuda': 'BMU',
 'bhutan': 'BTN',
 'bolivia': 'BOL',
 'bosnia and herzegovina': 'BIH',
 'botswana': 'BWA',
 'brazil': 'BRA',
 'british virgin islands': 'VGB',
 'brunei': 'BRN',
 'bulgaria': 'BGR',
 'burkina faso': 'BFA',
 'burma': 'MMR',
 'burundi': 'BDI',
 'cabo verde': 'CPV',
 'cambodia': 'KHM',
 'cameroon': 'CMR',
 'canada': 'CAN',
 'cayman islands': 'CYM',
 'central african republic': 'CAF',
 'chad': 'TCD',
 'chile': 'CHL',
 'china': 'CHN',
 'colombia': 'COL',
 'comoros': 'COM',
 'congo democratic': 'COD',
 'Congo republic': 'COG',
 'cook islands': 'COK',
 'costa rica': 'CRI',
 "cote d'ivoire": 'CIV',
 'croatia': 'HRV',
 'cuba': 'CUB',
 'curacao': 'CUW',
 'cyprus': 'CYP',
 'czech republic': 'CZE',
 'denmark': 'DNK',
 'djibouti': 'DJI',
 'dominica': 'DMA',
 'dominican republic': 'DOM',
 'ecuador': 'ECU',
 'egypt': 'EGY',
 'el salvador': 'SLV',
 'equatorial guinea': 'GNQ',
 'eritrea': 'ERI',
 'estonia': 'EST',
 'ethiopia': 'ETH',
 'falkland islands': 'FLK',
 'faroe islands': 'FRO',
 'fiji': 'FJI',
 'finland': 'FIN',
 'france': 'FRA',
 'french polynesia': 'PYF',
 'gabon': 'GAB',
 'gambia, the': 'GMB',
 'georgia': 'GEO',
 'germany': 'DEU',
 'ghana': 'GHA',
 'gibraltar': 'GIB',
 'greece': 'GRC',
 'greenland': 'GRL',
 'grenada': 'GRD',
 'guam': 'GUM',
 'guatemala': 'GTM',
 'guernsey': 'GGY',
 'guinea-bissau': 'GNB',
 'guinea': 'GIN',
 'guyana': 'GUY',
 'haiti': 'HTI',
 'honduras': 'HND',
 'hong kong': 'HKG',
 'hungary': 'HUN',
 'iceland': 'ISL',
 'india': 'IND',
 'indonesia': 'IDN',
 'iran': 'IRN',
 'iraq': 'IRQ',
 'ireland': 'IRL',
 'isle of man': 'IMN',
 'israel': 'ISR',
 'italy': 'ITA',
 'jamaica': 'JAM',
 'japan': 'JPN',
 'jersey': 'JEY',
 'jordan': 'JOR',
 'kazakhstan': 'KAZ',
 'kenya': 'KEN',
 'kiribati': 'KIR',
 'north korea': 'PRK',
 'south korea': 'KOR',
 'kosovo': 'KSV',
 'kuwait': 'KWT',
 'kyrgyzstan': 'KGZ',
 'laos': 'LAO',
 'latvia': 'LVA',
 'lebanon': 'LBN',
 'lesotho': 'LSO',
 'liberia': 'LBR',
 'libya': 'LBY',
 'liechtenstein': 'LIE',
 'lithuania': 'LTU',
 'luxembourg': 'LUX',
 'macau': 'MAC',
 'macedonia': 'MKD',
 'madagascar': 'MDG',
 'malawi': 'MWI',
 'malaysia': 'MYS',
 'maldives': 'MDV',
 'mali': 'MLI',
 'malta': 'MLT',
 'marshall islands': 'MHL',
 'mauritania': 'MRT',
 'mauritius': 'MUS',
 'mexico': 'MEX',
 'micronesia': 'FSM',
 'moldova': 'MDA',
 'monaco': 'MCO',
 'mongolia': 'MNG',
 'montenegro': 'MNE',
 'morocco': 'MAR',
 'mozambique': 'MOZ',
 'namibia': 'NAM',
 'nepal': 'NPL',
 'netherlands': 'NLD',
 'new caledonia': 'NCL',
 'new zealand': 'NZL',
 'nicaragua': 'NIC',
 'nigeria': 'NGA',
 'niger': 'NER',
 'niue': 'NIU',
 'northern mariana islands': 'MNP',
 'norway': 'NOR',
 'oman': 'OMN',
 'pakistan': 'PAK',
 'palau': 'PLW',
 'panama': 'PAN',
 'papua new guinea': 'PNG',
 'paraguay': 'PRY',
 'peru': 'PER',
 'philippines': 'PHL',
 'poland': 'POL',
 'portugal': 'PRT',
 'puerto rico': 'PRI',
 'qatar': 'QAT',
 'romania': 'ROU',
 'russia': 'RUS',
 'rwanda': 'RWA',
 'saint kitts and nevis': 'KNA',
 'saint lucia': 'LCA',
 'saint martin': 'MAF',
 'saint pierre and miquelon': 'SPM',
 'saint vincent and the grenadines': 'VCT',
 'samoa': 'WSM',
 'san marino': 'SMR',
 'sao tome and principe': 'STP',
 'saudi arabia': 'SAU',
 'senegal': 'SEN',
 'serbia': 'SRB',
 'seychelles': 'SYC',
 'sierra leone': 'SLE',
 'singapore': 'SGP',
 'sint maarten': 'SXM',
 'slovakia': 'SVK',
 'slovenia': 'SVN',
 'solomon islands': 'SLB',
 'somalia': 'SOM',
 'south africa': 'ZAF',
 'south sudan': 'SSD',
 'spain': 'ESP',
 'sri lanka': 'LKA',
 'sudan': 'SDN',
 'suriname': 'SUR',
 'swaziland': 'SWZ',
 'sweden': 'SWE',
 'switzerland': 'CHE',
 'syria': 'SYR',
 'taiwan': 'TWN',
 'tajikistan': 'TJK',
 'tanzania': 'TZA',
 'thailand': 'THA',
 'timor-leste': 'TLS',
 'togo': 'TGO',
 'tonga': 'TON',
 'trinidad and tobago': 'TTO',
 'tunisia': 'TUN',
 'turkey': 'TUR',
 'turkmenistan': 'TKM',
 'tuvalu': 'TUV',
 'uganda': 'UGA',
 'ukraine': 'UKR',
 'united arab emirates': 'ARE',
 'united kingdom': 'GBR',
 'united states': 'USA',
 'uruguay': 'URY',
 'uzbekistan': 'UZB',
 'vanuatu': 'VUT',
 'venezuela': 'VEN',
 'vietnam': 'VNM',
 'virgin islands': 'VGB',
 'west bank': 'WBG',
 'yemen': 'YEM',
 'zambia': 'ZMB',
 'zimbabwe': 'ZWE'}

## countries 
from collections import Counter
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]
    
def geoplot(ddf):
    country_with_code, country = {}, {}
    shows_countries = ", ".join(ddf['country'].dropna()).split(", ")
    for c,v in dict(Counter(shows_countries)).items():
        code = ""
        if c.lower() in country_codes:
            code = country_codes[c.lower()]
        country_with_code[code] = v
        country[c] = v

    data = [dict(
            type = 'choropleth',
            locations = list(country_with_code.keys()),
            z = list(country_with_code.values()),
            colorscale = [[0,"rgb(5, 10, 172)"],[0.65,"rgb(40, 60, 190)"],[0.75,"rgb(70, 100, 245)"],\
                        [0.80,"rgb(90, 120, 245)"],[0.9,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
            autocolorscale = False,
            reversescale = True,
            marker = dict(
                line = dict (
                    color = 'gray',
                    width = 0.5
                ) ),
            colorbar = dict(
                autotick = False,
                title = ''),
          ) ]

    layout = dict(
        title = '',
        geo = dict(
            showframe = False,
            showcoastlines = False,
            projection = dict(
                type = 'Mercator'
            )
        )
    )

    fig = dict( data=data, layout=layout )
    iplot( fig, validate=False, filename='d3-world-map' )
    return country

country_vals = geoplot(datafr)
tabs = Counter(country_vals).most_common(25)

labels = [_[0] for _ in tabs][::-1]
values = [_[1] for _ in tabs][::-1]
trace1 = go.Bar(y=labels, x=values, orientation="h", name="", marker=dict(color="#a678de"))

data = [trace1]
layout = go.Layout(title="Countries with most content", height=700, legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
d1 = datafr[datafr["type"] == "TV Show"]
d2 = datafr[datafr["type"] == "Movie"]

In [ ]:
col = "year_added"
vc1 = d1[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = d2[col].value_counts().reset_index()
vc2 = vc2.rename(columns = {col : "count", "index" : col})
vc2['percent'] = vc2['count'].apply(lambda x : 100*x/sum(vc2['count']))
vc2 = vc2.sort_values(col)

In [ ]:
trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Bar(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))
data = [trace1, trace2]
layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
col = "release_year"

vc1 = d1[col].value_counts().reset_index()
vc1 = vc1.rename(columns = {col : "count", "index" : col})
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = d2[col].value_counts().reset_index()
vc2 = vc2.rename(columns = {col : "count", "index" : col})
vc2['percent'] = vc2['count'].apply(lambda x : 100*x/sum(vc2['count']))
vc2 = vc2.sort_values(col)

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Bar(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))
data = [trace1, trace2]
layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

In [ ]:
import plotly.figure_factory as ff
d2['duration'] = d2['duration'].replace({'min':''}, regex=True)
x1 = d2['duration'].fillna(0.0).astype(float)
fig = ff.create_distplot([x1], ['a'], bin_size=0.7, curve_type='normal', colors=["#6ad49b"])
fig.update_layout(title_text='Distplot with Normal Distribution')
fig.show()

In [ ]:
titles_by_country= datafr.groupby('country')[['title']].sum()
titles_by_country_label= datafr.groupby('country')[['title']].sum().index.get_level_values(0)

In [ ]:
print(titles_by_country_label)

Index([', France, Algeria', ', South Korea', 'Argentina',
       'Argentina, Brazil, France, Poland, Germany, Denmark',
       'Argentina, Chile', 'Argentina, Chile, Peru', 'Argentina, France',
       'Argentina, France, United States, Germany, Qatar', 'Argentina, Italy',
       'Argentina, Spain',
       ...
       'Uruguay, Argentina, Germany, Spain', 'Uruguay, Argentina, Spain',
       'Uruguay, Germany', 'Uruguay, Guatemala', 'Uruguay, Spain, Mexico',
       'Venezuela', 'Venezuela, Colombia', 'Vietnam', 'West Germany',
       'Zimbabwe'],
      dtype='object', name='country', length=748)


In [ ]:
datafr['cast'] = datafr['cast'].fillna('')
datafr['director'] = datafr['director'].fillna('')

First we want to try a simple Content Based Recommendor System which will recommend us movies/series which are similar to each other. To achieve this, we will compute pairwise cosine similarity scores for all movies/series based on their genre which is given by the listed_in feature and recommend movies/series based on that similarity score threshold.

Since the data in question is in Natural Language, we will have to compute the word vectors for the data and find the similarity between each vector. We will be using a built-in class called as Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each document. The TF-IDF score gives us the frequency of words occuring in a document which helps us evaluating the similarity between two documents.

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(datafr['listed_in'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(8807, 44)

Using pairwise Linear Kernel to find similarities in the Genres(listed_in) attribute, 

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(8807, 8807)

In [ ]:
cosine_sim[1]

array([0.        , 1.        , 0.62052765, ..., 0.        , 0.        ,
       0.11412729])

In [ ]:
indices = pd.Series(datafr.index, index=datafr['title']).drop_duplicates()
indices[:10]

title
Dick Johnson Is Dead                0
Blood & Water                       1
Ganglands                           2
Jailbirds New Orleans               3
Kota Factory                        4
Midnight Mass                       5
My Little Pony: A New Generation    6
Sankofa                             7
The Great British Baking Show       8
The Starling                        9
dtype: int64

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:10]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return datafr['title'].iloc[movie_indices]

In [ ]:
get_recommendations("Kota Factory")

47                The Smart Money Woman
121                      Hotel Del Luna
186                Hometown Cha-Cha-Cha
287                      I Need Romance
316                        Office Girls
319    The Prince Who Turns into a Frog
321                         Two Fathers
322                   You're My Destiny
397                     Feels Like Ishq
Name: title, dtype: object

In [ ]:
data = {'model': cosine_sim,"tfidf": tfidf,"tfidf_matrix":tfidf_matrix,"get_recommendations":get_recommendations,"indices":indices}

In [ ]:
with open('saved_steps.pkl','wb') as file:
  pickle.dump(data,file)

In [ ]:
with open('saved_steps.pkl', 'rb') as file:
  data =pickle.load(file)
cosine_sim_loaded = data["model"]
tfidf_loaded = data["tfidf"]
tfidf_matrix_loaded = data["tfidf_matrix"]
get_recommendations_loaded = data["get_recommendations"]
indices_loaded = data["indices"]

This system has given us some good recommendations based on the genre associated with the title but let us try to make it better. There is a high possibility that we would like a recommendation for a movie/series by the same actor or director or of similar genres. For the next model let's try to combine all of these features to make our recommendation algorithm better.

To combine these features together, we need to concatenate them together and create a soup feature. To do that we need to clean the data. Since we are considering the name of Actors and Directors, we need to make sure that actors with same names should not be considered the same. For example, John Travolta and John Krasinski are two separate actors, to resolve that we need to remove the space between them.



In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
# Apply clean_data function to your features.
features = ['cast', 'director', 'listed_in']

for feature in features:
    datafr[feature] = datafr[feature].apply(clean_data)

In [ ]:
#Define a function to create a soup feature
def create_soup(x):
    return  ''.join(x['cast']) + ',' + ''.join(x['director'])  + ',' + ''.join(x['listed_in']) 

# Create a new soup feature
datafr['soup'] = datafr.apply(create_soup, axis=1)
datafr['soup']

0                           ,kirstenjohnson,documentaries
1       amaqamata,khosingema,gailmabalane,thabangmolab...
2       samibouajila,tracygotoas,samueljouy,nabihaakka...
3                    ,audreyestrougo,docuseries,realitytv
4       mayurmore,jitendrakumar,ranjanraj,alamkhan,ahs...
                              ...                        
8802    markruffalo,jakegyllenhaal,robertdowneyjr.,ant...
8803                   ,,kids'tv,koreantvshows,tvcomedies
8804    jesseeisenberg,woodyharrelson,emmastone,abigai...
8805    timallen,courteneycox,chevychase,katemara,ryan...
8806    vickykaushal,sarah-janedias,raaghavchanana,man...
Name: soup, Length: 8807, dtype: object

In [ ]:
#Define a Count Vectorizer Object. Remove all english stop words such as 'the', 'a'
count = CountVectorizer(stop_words='english')

In [ ]:
count_matrix = count.fit_transform(datafr['soup'])
count_matrix.shape

(8807, 41871)

In [ ]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
datafr = datafr.reset_index()
indices = pd.Series(datafr.index, index=datafr['title'])

In [ ]:
get_recommendations("Dick Johnson Is Dead", cosine_sim2 )

404                9to5: The Story of a Movement
854                 Creating an Army of the Dead
1011                                Free to Play
1161    Elizabeth and Margaret: Love and Loyalty
1425                 Creating The Queen's Gambit
1466                The Minimalists: Less Is Now
2608    Murder to Mercy: The Cyntoia Brown Story
2779          Crip Camp: A Disability Revolution
5052        Liberated: The New Sexual Revolution
Name: title, dtype: object

In [ ]:
data = {'model: cosine_'}